Parquet是一种专为大数据处理设计的开源列式存储文件格式。与传统的行式存储格式（如CSV）不同，Parquet将数据按列而不是按行存储。这种设计使得Parquet在查询特定列时具有显著的性能优势，因为它只需读取所需的列，而无需加载整个数据表，这大大减少了I/O操作和内存占用。

Parquet格式的几个关键特点包括：

1. **高效压缩**：由于数据按列存储，Parquet能够更有效地压缩数据，通常可以减少75%的存储空间。这种压缩不仅减少了存储需求，还加速了数据的读取和处理。

2. **复杂数据类型支持**：Parquet支持嵌套的数据结构和复杂的数据类型，这使其特别适合处理大规模、结构复杂的数据集。

3. **分区和并行处理**：Parquet文件可以被分割成更小的块，这使得在分布式计算框架（如Apache Spark和Hadoop）中可以并行处理，进一步提高了处理效率。

4. **模式演进**：Parquet支持在不破坏现有数据的情况下对数据模式进行修改，比如添加新的列，这使得数据架构能够随着业务需求的发展而灵活演变。

总体而言，Parquet格式因其高效的存储和查询性能，特别适用于大数据分析场景，广泛应用于数据湖、机器学习管道和实时分析等领域。

### 下载FineTome-100k数据集（参考其格式）

In [2]:
from datasets import load_dataset

# 加载mlabonne/FineTome-100k数据集的训练部分，并将其下载到当前文件夹下。
dataset = load_dataset("mlabonne/FineTome-100k", split="train")

# 保存数据集为parquet格式
dataset.to_parquet("./FineTome-100k.parquet")

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/100 [00:00<?, ?ba/s]

258074273

### 查看FineTome-100k数据集格式细节

In [9]:
import pandas as pd
import json
from pandas import json_normalize

# 加载parquet文件
df = pd.read_parquet("./FineTome-100k.parquet")

# 1. 查看数据的基本信息
print("数据框信息:")
print(df.info())

# 2. 查看数据集的前几行内容
print("\n数据框的前5行内容:")
print(df.head())

# 3. 查看数据集的列名称和类型
print("\n列名称和数据类型:")
print(df.dtypes)

# 4. 检查数据中的缺失值
print("\n每列中的缺失值数量:")
print(df.isnull().sum())

# 5. 遍历所有列并输出每一列的第一条记录
print("\n每列的示例数据:")
for col in df.columns:
    print(f"列名: {col}")
    print(df[col].iloc[0])
    print("\n")

# 6. 如果某些列包含嵌套JSON结构，解析并展开这些结构（以“conversations”列为例）
if 'conversations' in df.columns:
    print("\n解析 'conversations' 列:")
    # 假设“conversations”列包含嵌套JSON结构
    nested_json = df['conversations'].iloc[0]
    # 如果是字符串形式的JSON，需要先转换
    if isinstance(nested_json, str):
        nested_json = json.loads(nested_json)
    
    # 展开JSON
    normalized_data = json_normalize(nested_json)
    print("'conversations' 列的规范化数据:")
    print(normalized_data.head())

# 7. 统计对话内容的长度
if 'conversations' in df.columns:
    df['text_length'] = df['conversations'].apply(lambda x: len(str(x)))
    print("\n'conversations' 列中文本长度的统计信息:")
    print(df['text_length'].describe())

# 8. 手动查看前5条记录的具体内容
print("\n'conversations' 列中的前5条记录:")
for i in range(5):
    print(f"记录 {i}:")
    print(df['conversations'].iloc[i])
    print("\n")

# 9. 检查数据类型是否符合预期
print("\n检查数据类型:")
print(df.dtypes)

数据框信息:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   conversations  100000 non-null  object 
 1   source         100000 non-null  object 
 2   score          100000 non-null  float64
dtypes: float64(1), object(2)
memory usage: 2.3+ MB
None

数据框的前5行内容:
                                       conversations  \
0  [{'from': 'human', 'value': 'Explain what bool...   
1  [{'from': 'human', 'value': 'Explain how recur...   
2  [{'from': 'human', 'value': 'Explain what bool...   
3  [{'from': 'human', 'value': 'Explain the conce...   
4  [{'from': 'human', 'value': 'Print the reverse...   

                     source     score  
0  infini-instruct-top-500k  5.212621  
1  infini-instruct-top-500k  5.157649  
2  infini-instruct-top-500k  5.147540  
3  infini-instruct-top-500k  5.053656  
4  infini-instruct-top-500k  5.045648  

列名称和数据类型:
co

以下是对 `FineTome-100k` 数据集的结构和内容参数的分析汇总：

#### 1. 数据框信息
- **数据集大小**：共 100,000 条记录。
- **列数**：总共有 3 列。
- **数据类型**：`conversations` 和 `source` 列的数据类型为 `object`，`score` 列的数据类型为 `float64`。

#### 2. 数据框的前5行内容
- **conversations**：每条记录包含一组对话，格式为 JSON 数组，其中包含“from”（发信方，如 "human" 或 "gpt"）和“value”（对话内容）的键值对。
  - **`from`**: 消息的发送者，例如 `"human"` 或 `"gpt"`。
  - **`value`**: 实际的消息内容，是一段文本，通常是提问或回答。
- **source**：每条记录的数据来源，均为 `infini-instruct-top-500k`。
- **score**：每条记录对应的得分，表示该对话的质量或其他评分指标。

#### 3. 列名称和数据类型
- **conversations**：`object` 类型，包含对话的详细内容。
- **source**：`object` 类型，表示数据来源。
- **score**：`float64` 类型，表示对每条记录的评分。
- **text_length**：这是计算出的列，表示 `conversations` 列中对话文本的长度（字符数），类型为 `int64`。

#### 4. 每列中的缺失值数量
- 所有列 (`conversations`, `source`, `score`) 中均无缺失值，表明数据是完整的。

#### 5. 每列的示例数据
- **conversations**：包含嵌套的 JSON 结构，记录了人类用户和 GPT 之间的对话内容。这些对话内容详细而复杂，涉及编程、逻辑操作、递归等话题。
- **source**：数据来源为 `infini-instruct-top-500k`，暗示数据可能是从一个大型指令式数据集衍生出来的。
- **score**：分数范围通常在 5.0 左右，可能表示对对话质量的评估。

每一行代表一条对话记录，其中 `conversations` 字段可能包含多条交互消息。例如：

```python
conversations = [
    {"from": "human", "value": "Explain what boolean operators are, what they do, and provide examples of how they can be used in programming."},
    {"from": "gpt", "value": "Boolean operators are logical operators used in programming to manipulate boolean values..."}
]
source = "infini-instruct-top-500k"
score = 5.212621
```

#### 6. 解析 'conversations' 列
- 对 `conversations` 列中的 JSON 结构进行了展开，显示了 `from` 和 `value` 两个字段。`from` 标识消息的发送者 (`human` 或 `gpt`)，`value` 是对应的文本内容。

#### 7. 'conversations' 列中文本长度的统计信息
- **平均长度**：2629 个字符
- **标准差**：2537 个字符
- **最小长度**：206 个字符
- **最大长度**：68253 个字符
- 这表明 `conversations` 列中的对话内容长度差异较大。

#### 8. 'conversations' 列中的前5条记录
- 展示了前5条记录中的详细对话内容，包括编程示例、递归解释、布尔运算符的使用等。

#### 9. 检查数据类型
- 数据类型与预期一致：
  - `conversations`: 对象类型（`object`），通常是包含对话的列表。
  - `source`: 对象类型（`object`），表示数据的来源。
  - `score`: 浮点数类型（`float64`），表示评分。

### 将input.csv转换为Parquet格式

In [12]:
import pandas as pd
import json

# 加载CSV文件
df = pd.read_csv('input.csv')

# 定义一个函数，将每一行数据转换为目标格式的JSON结构
def convert_to_conversation(row):
    # 返回一个 Python 字典列表，而不是 JSON 字符串
    return [
        {"from": "human", "value": row['human']},
        {"from": "gpt", "value": row['gpt']}
    ]

# 应用转换函数，生成新的 'conversations' 列
df['conversations'] = df.apply(convert_to_conversation, axis=1)

# 保留需要的列，丢弃原始列
df = df[['conversations', 'source', 'score']]

# 检查score列的数据类型，并将其转换为float64
df['score'] = df['score'].astype('float64')

# 保存为Parquet文件
df.to_parquet('output.parquet', index=False)

print("CSV文件已成功转换为Parquet文件，并保存为'output.parquet'。")

CSV文件已成功转换为Parquet文件，并保存为'output.parquet'。


### 检查output.parquet格式

In [15]:
import pandas as pd
import json
from pandas import json_normalize

# 加载parquet文件
df = pd.read_parquet("./output.parquet")

# 1. 查看数据的基本信息
print("数据框信息:")
print(df.info())

# 2. 查看数据集的前几行内容
print("\n数据框的前5行内容:")
print(df.head())

# 3. 查看数据集的列名称和类型
print("\n列名称和数据类型:")
print(df.dtypes)

# 4. 检查数据中的缺失值
print("\n每列中的缺失值数量:")
print(df.isnull().sum())

# 5. 遍历所有列并输出每一列的第一条记录
print("\n每列的示例数据:")
for col in df.columns:
    print(f"列名: {col}")
    print(df[col].iloc[0])
    print("\n")

# 6. 如果某些列包含嵌套JSON结构，解析并展开这些结构（以“conversations”列为例）
if 'conversations' in df.columns:
    print("\n解析 'conversations' 列:")
    # 假设“conversations”列包含嵌套JSON结构
    nested_json = df['conversations'].iloc[0]
    # 如果是字符串形式的JSON，需要先转换
    if isinstance(nested_json, str):
        nested_json = json.loads(nested_json)
    
    # 展开JSON
    normalized_data = json_normalize(nested_json)
    print("'conversations' 列的规范化数据:")
    print(normalized_data.head())

# 7. 统计对话内容的长度
if 'conversations' in df.columns:
    df['text_length'] = df['conversations'].apply(lambda x: len(str(x)))
    print("\n'conversations' 列中文本长度的统计信息:")
    print(df['text_length'].describe())

# 8. 手动查看前5条记录的具体内容
print("\n'conversations' 列中的前5条记录:")
for i in range(5):
    print(f"记录 {i}:")
    print(df['conversations'].iloc[i])
    print("\n")

# 9. 检查数据类型是否符合预期
print("\n检查数据类型:")
print(df.dtypes)

数据框信息:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876 entries, 0 to 875
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   conversations  876 non-null    object 
 1   source         876 non-null    object 
 2   score          876 non-null    float64
dtypes: float64(1), object(2)
memory usage: 20.7+ KB
None

数据框的前5行内容:
                                       conversations     source  score
0  [{'from': 'human', 'value': 'What is the origi...  FF-Studio    5.0
1  [{'from': 'human', 'value': '请问这个模型的开发者是谁？'}, ...  FF-Studio    5.0
2  [{'from': 'human', 'value': 'Who made this mod...  FF-Studio    5.0
3  [{'from': 'human', 'value': '这个模型的背后团队是谁？'}, {...  FF-Studio    5.0
4  [{'from': 'human', 'value': 'Where did this mo...  FF-Studio    5.0

列名称和数据类型:
conversations     object
source            object
score            float64
dtype: object

每列中的缺失值数量:
conversations    0
source           0
score            0
dtyp